In [ ]:
%matplotlib inline

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# digitsデータを使うためにimportする
from sklearn.datasets import load_digits

## GPUを使える場合はGPUを使うための準備

In [ ]:
# GPUを使える場合はGPUを使うための準備
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)

## データの読み込み、Xとy別々に保存

In [ ]:
X = load_digits()["data"]
y = load_digits()["target"]

In [ ]:
# Xに格納されているデータを確認
X.shape

In [ ]:
# yに格納されているデータを確認
y.shape

In [ ]:
idx = 2
plt.imshow(X[idx].reshape(8,8), cmap=plt.cm.gray_r, interpolation='nearest')
print("Target: ", y[idx])

## NumPyのarrayをtorch.tensor型に変換する
---
torch.tensor型とは、多次元配列を扱うためのデータ構造です。<br>
Numpyのndarrayとほぼ同様の操作ができますが、torch.tensor型はNumpyと違いGPU処理が可能です。

In [ ]:
X_tensor = torch.tensor(X, dtype=torch.float64)
y_tensor = torch.tensor(y, dtype=torch.int64)

In [ ]:
print(X_tensor)
print(y_tensor)

## ネットワークの書き方　nn.Sequentialを使う場合の書き方
---
nn.Sequentialは、"層"を積み重ねてネットワークを構築する際に利用します。

In [ ]:
net = nn.Sequential(
    nn.Linear(64,32),
    nn.ReLU(),
    nn.Linear(32,10)
)

In [ ]:
net.parameters

## GPUに転送
---
GPUはCPUと違って、計算処理を順番に行うのではなく、並行化して行うことができます。<br>
Deep Learningでは膨大な量の行列計算を行うので、GPUを利用し並列化して計算を行うと高速化が見込めます。

In [ ]:
X_tensor = X_tensor.to(device)
y_tensor = y_tensor.to(device)
net = net.to(device, torch.double)

## 目的関数の設定を行う
---
多クラス分類問題なので交差エントロピー誤差関数を設定します。<br>
※ 回帰問題のために、MSELoss()等も用意されています。[Loss functionドキュメント](https://pytorch.org/docs/stable/nn.html#loss-functions)<br>
※[nn.CrossEntropyLoss()](https://pytorch.org/docs/stable/nn.html#crossentropyloss)は、「Softmax関数」と「負の対数尤度損失」を結合しています。

In [ ]:
loss_func = nn.CrossEntropyLoss()
print(loss_func)

## Optimizerを定義する
---
今回はSGDを使用。<br>
Optimizerには、Adam等も用意されている。[Optimizerドキュメント](https://pytorch.org/docs/stable/optim.html)

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.1)

## 学習

In [ ]:
loss_track = []
for epoc in range(500):
    # まず勾配の初期化
    optimizer.zero_grad()
    
    # フォワード計算
    y_pred = net(X_tensor)
    
    # 損失の計算
    loss = loss_func(y_pred, y_tensor)
    
    # 誤差逆伝播法を行い勾配の計算
    loss.backward()
    
    # パラメーターの更新
    optimizer.step()
    
    # 交差エントロピー誤差をトラッキング
    loss_track.append(loss)

## 交差エントロピー誤差の推移

In [ ]:
plt.plot(loss_track)